In [1]:
!pip install transformers
!pip install PersianStemmer
!pip install https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip --upgrade
!pip install hazm
!pip install demoji

  Using cached https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip


In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0, '/content/drive/My Drive/modules')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score,precision_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import preprocess
from statistics import mean
import demoji
demoji.download_codes()

... OK (Got response in 0.18 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [4]:
def run_svm():
    SVM_model.fit(x_train, y_train)
    prediction_SVM = SVM_model.predict(x_test)
    f_score = round(((f1_score(y_test, prediction_SVM)) * 100), 2)
    accuracy = round((accuracy_score(y_test, prediction_SVM) * 100), 2)
    precision = round((precision_score(y_test, prediction_SVM) * 100), 2)
    recall = round((recall_score(y_test, prediction_SVM) * 100), 2)
    return f_score, accuracy, precision, recall

In [6]:
stop_words = 'custom'
stemming = True
dataset = preprocess.auto_preprocess(pd.read_csv('/content/drive/MyDrive/data/initial_dataset.csv'), stop_words_type=stop_words, stemming=stemming)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

custom stop words selected
there are 5 outlier in data
current dataframe length before remove 5 outlier sentence is: 2000
current dataframe length after remove 5 outlier sentence is: 1995
auto preprocess completed done on dataframe!
Enjoy!


In [7]:
vectorizer = CountVectorizer(ngram_range=(1,3), min_df=4, max_df=0.8)
x = vectorizer.fit_transform(dataset['sentence'])
y = (dataset['label'].to_numpy())
SVM_model = svm.SVC(random_state=42)
params = {'C': 50, 'class_weight': {0: 1, 1: 4}, 'gamma': 'auto', 'kernel': 'sigmoid'} 
SVM_model.set_params(**params)

f1_list = []
accuracy_list = []
precision_list = []
recall_list = []

for train_index, test_index in skf.split(dataset['sentence'], dataset['label']):

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    f1, accuracy, precision, recall = run_svm()
    f1_list.append(round(f1, 2))
    accuracy_list.append(round(accuracy, 2))
    precision_list.append(round(precision, 2))
    recall_list.append(round(recall, 2))

print (f"f1 is: {mean(f1_list)}\naccuracy is: {mean(accuracy_list)}\nprecision is: {mean(precision_list)}\nrecall is: {mean(recall_list)}")      

f1 is: 72.89
accuracy is: 91.278
precision is: 78.164
recall is: 68.532
